# Module 6 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-02-programming.ipynb
    +--module-02-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

# Unification

This is actually Part I of a two part assignment. Next week, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

In [1]:
import tokenize
import cStringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm). The first function is the `atom` function.

In [2]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [3]:
def parse( exp):
    src = cStringIO.StringIO( exp).readline
    tokens = tokenize.generate_tokens( src)
    return atom( tokens.next, tokens.next())

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [4]:
parse("Fred")

'Fred'

In [5]:
parse( "?x")

'?x'

In [6]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [7]:
parse( "(father-of Barney (son-of Barney))")

['father', '-', 'of', 'Barney', ['son', '-', 'of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return None (if unification completely fails), {} (the empty substitution list) or a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [8]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [9]:
is_variable( "Fred")

False

In [10]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [11]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [12]:
is_constant( "Fred")

True

In [13]:
is_constant( "?fred")

False

In [14]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>

You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. You'll need to fix this next week.)

-----

Helper 1 documentation here.

In [15]:
def helper1():
    pass

-----
(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


In [16]:
def unification( list_expression1, list_expression2):
    pass

In [17]:
def unify( s_expression1, s_expression2):
    return unification( parse( s_expression1), parse( s_expression2))

Using `unify`, solve the problems in the self-check. **Important** use `unify` **not** `unification`. Do one expression per cell: